In [ ]:
import pandas as pd

df = pd.read_excel("Companies_with_Risk.xlsx")

df['FiscalDate'] = pd.to_datetime(df['FiscalDate'], errors='coerce')

exclude_cols = ['GST', 'CropTax%', 'Inflation%', 'RepoRate%', 'USDINR_Close']

def handle_outliers_within_company(group):
    for col in group.select_dtypes(include=['float64', 'int64']).columns:
        if col not in exclude_cols:
            Q1 = group[col].quantile(0.25)
            Q3 = group[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            group[col] = group[col].clip(lower=lower_bound, upper=upper_bound)
    return group

# Apply outlier handling company-wise
df = df.groupby("Company", group_keys=False).apply(handle_outliers_within_company)

# Drop exact duplicate rows (if any)
df = df.drop_duplicates()

# Sorting
df = df.sort_values(by=["Company", "FiscalDate"], ascending=[True, True]).reset_index(drop=True)

# Save cleaned dataset
df.to_excel("Companies_with_Risk_Cleaned.xlsx", index=False)
print("Cleaned and sorted dataset saved as Companies_with_Risk_Cleaned.xlsx")


/tmp/ipython-input-3123634079.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("Company", group_keys=False).apply(handle_outliers_within_company)


Cleaned and sorted dataset saved as Companies_with_Risk_Cleaned.xlsx


In [ ]:
import numpy as np

# Ensure FiscalDate is datetime
df["FiscalDate"] = pd.to_datetime(df["FiscalDate"])
df = df.sort_values(by=["Company", "FiscalDate"])

# ==========================
# Feature Engineering
# ==========================
df["Revenue_Growth"] = df.groupby("Company")["Revenue"].pct_change()
df["Expense_Growth"] = df.groupby("Company")["Expenses"].pct_change()
df["Profit_Growth"] = df.groupby("Company")["Net Profit"].pct_change()
df["Expense_to_Revenue"] = df["Expenses"] / df["Revenue"]

# Replace NaN with 0 (first row of each company)
df[["Revenue_Growth", "Expense_Growth", "Profit_Growth"]] = df[["Revenue_Growth", "Expense_Growth", "Profit_Growth"]].fillna(0)

# ==========================
# Standardize (z-scores per company)
# ==========================
features = ["Revenue_Growth","Expense_Growth","Profit_Growth","Expense_to_Revenue",
            "GST","Inflation%","RepoRate%","USDINR_Close"]

for col in features:
    df[col + "_z"] = df.groupby("Company")[col].transform(
        lambda x: (x - x.mean()) / x.std(ddof=0) if x.std(ddof=0) != 0 else 0
    )

# ==========================
# Weighted Risk Score
# ==========================
df["Risk_Score"] = (
    -df["Revenue_Growth_z"] +        # Lower revenue growth = higher risk
     df["Expense_Growth_z"] +        # Higher expense growth = higher risk
    -df["Profit_Growth_z"] +         # Lower profit growth = higher risk
     df["Expense_to_Revenue_z"] +    # Higher expense ratio = higher risk
     df["GST_z"] +
     df["Inflation%_z"] +
     df["RepoRate%_z"] +
     df["USDINR_Close_z"]
)

# ==========================
# Classify Risk Levels
# ==========================
def classify_risk(x):
    if x >= 1:   # Above 1 std deviation = stressed
        return 2  # High risk
    elif x <= -0.5:
        return 0  # Low risk
    else:
        return 1  # Medium risk

df["Risk"] = df.groupby("Company")["Risk_Score"].transform(lambda x: x.apply(classify_risk))

# ==========================
# Save with new Risk column
# ==========================
output_path = "/content/Companies_risklabel.xlsx"
df.to_excel(output_path, index=False)

output_path


/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:52: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


'/content/Companies_risklabel.xlsx'

In [ ]:
df = pd.read_excel("/content/Companies_risklabel.xlsx")
df["FiscalDate"] = pd.to_datetime(df["FiscalDate"])
df = df.sort_values(["Company", "FiscalDate"]).reset_index(drop=True)

# Function to compute growth metrics
def compute_growth(group):
    group = group.sort_values("FiscalDate").reset_index(drop=True)
    group["Year"] = group["FiscalDate"].dt.year

    freq = group["Year"].value_counts().max()
    is_quarterly = freq > 1

    if is_quarterly:
        # Quarterly companies
        group["Revenue_QoQ_Growth"] = group["Revenue"].pct_change(1) * 100
        group["Revenue_YoY_Growth"] = group["Revenue"].pct_change(4) * 100

        if "EPS" in group.columns:
            group["EPS_QoQ_Growth"] = group["EPS"].pct_change(1) * 100
            group["EPS_YoY_Growth"] = group["EPS"].pct_change(4) * 100
        else:
            group["EPS_QoQ_Growth"] = None
            group["EPS_YoY_Growth"] = None

        group["Frequency"] = "Quarterly"

    else:
        # Yearly companies
        group["Revenue_QoQ_Growth"] = 0
        group["Revenue_YoY_Growth"] = group["Revenue"].pct_change(1) * 100
        if "EPS" in group.columns:
            group["EPS_QoQ_Growth"] = 0
            group["EPS_YoY_Growth"] = group["EPS"].pct_change(1) * 100
        else:
            group["EPS_QoQ_Growth"] = 0
            group["EPS_YoY_Growth"] = None

        group["Frequency"] = "Yearly"

    return group

# Apply per company
result = df.groupby("Company", group_keys=False).apply(compute_growth)

# --- Yearly Aggregation for Quarterly Companies ---
yearly_agg = (
    result[result["Frequency"] == "Quarterly"]
    .groupby(["Company", "Year"], as_index=False)
    .agg({
        "Revenue": "sum",
        "EPS": "sum" if "EPS" in result.columns else "mean"
    })
)

# Mark incomplete years (less than 4 quarters)
counts = (
    result[result["Frequency"] == "Quarterly"]
    .groupby(["Company", "Year"])["FiscalDate"].count()
    .reset_index(name="Quarters_Count")
)
yearly_agg = yearly_agg.merge(counts, on=["Company", "Year"], how="left")
yearly_agg["Incomplete_Year"] = yearly_agg["Quarters_Count"] < 4

# Compute YoY growth on yearly totals
yearly_agg["Revenue_YoY_Growth"] = yearly_agg.groupby("Company")["Revenue"].pct_change(1) * 100
if "EPS" in result.columns:
    yearly_agg["EPS_YoY_Growth"] = yearly_agg.groupby("Company")["EPS"].pct_change(1) * 100
else:
    yearly_agg["EPS_YoY_Growth"] = None
yearly_agg["Frequency"] = "Quarterly_Aggregated"

# --- Fill NaN values with 0 for ML ---
for col in ["Revenue_QoQ_Growth", "Revenue_YoY_Growth", "EPS_QoQ_Growth", "EPS_YoY_Growth"]:
    if col in result.columns:
        result[col] = result[col].fillna(0)

# Save outputs
output_file = "Companies_risklabel_with_growth.xlsx"
result.to_excel(output_file, index=False)


/tmp/ipython-input-3999056165.py:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby("Company", group_keys=False).apply(compute_growth)


In [ ]:
file_path = "/content/Companies_risklabel_with_growth.xlsx"
df = pd.read_excel(file_path)


# 1. EBITDA Margin (%)
df["EBITDA_Margin_%"] = (df["EBITDA"] / df["Revenue"].replace(0, pd.NA)) * 100

# 2. Net Profit Margin (%)
df["Net_Profit_Margin_%"] = (df["Net Profit"] / df["Revenue"].replace(0, pd.NA)) * 100

# 3. Interest Coverage Ratio (x times)
df["Interest_Coverage"] = df["EBITDA"] / df["Interest"].replace(0, pd.NA)

# 4. PBT / Interest Ratio (x times)
df["PBT_Interest_Ratio"] = df["PBT"] / df["Interest"].replace(0, pd.NA)

# 5. Debt Proxy (Interest / Revenue) %
df["Debt_Proxy_%"] = (df["Interest"] / df["Revenue"].replace(0, pd.NA)) * 100

# --- Save to new Excel file ---
output_path = "/content/Companies_with_Derived_Metrics.xlsx"
df.to_excel(output_path, index=False)

output_path

'/content/Companies_with_Derived_Metrics.xlsx'

In [ ]:
file_path_updated = "/content/Companies_with_Derived_Metrics.xlsx"
df = pd.read_excel(file_path_updated)
df = df.sort_values(by=["Company", "FiscalDate"])

# 1. Stability Metrics (Rolling Volatility of QoQ Growth, 4 quarters)


# Rolling volatility of Revenue QoQ Growth
df["Revenue_QoQ_Volatility"] = (
    df.groupby("Company")["Revenue_QoQ_Growth"]
      .rolling(window=4, min_periods=2)
      .std()
      .reset_index(level=0, drop=True)
)

# Rolling volatility of EPS QoQ Growth
df["EPS_QoQ_Volatility"] = (
    df.groupby("Company")["EPS_QoQ_Growth"]
      .rolling(window=4, min_periods=2)
      .std()
      .reset_index(level=0, drop=True)
)


# 2. Sector-relative Metrics


# Median EBITDA Margin % by Sector & FiscalDate
df["Sector_Median_EBITDA_Margin"] = (
    df.groupby(["Sector", "FiscalDate"])["EBITDA_Margin_%"]
      .transform("median")
)

# Margin gap vs sector median
df["EBITDA_Margin_Gap_vs_Sector"] = (
    df["EBITDA_Margin_%"] - df["Sector_Median_EBITDA_Margin"]
)

# Median Revenue QoQ Growth by Sector & FiscalDate
df["Sector_Median_Revenue_QoQ_Growth"] = (
    df.groupby(["Sector", "FiscalDate"])["Revenue_QoQ_Growth"]
      .transform("median")
)

# Revenue growth gap vs sector median
df["Revenue_Growth_Gap_vs_Sector"] = (
    df["Revenue_QoQ_Growth"] - df["Sector_Median_Revenue_QoQ_Growth"]
)


# Handle Missing Values

df = df.fillna({
    "Revenue_QoQ_Volatility": 0,
    "EPS_QoQ_Volatility": 0,
    "Sector_Median_EBITDA_Margin": 0,
    "EBITDA_Margin_Gap_vs_Sector": 0,
    "Sector_Median_Revenue_QoQ_Growth": 0,
    "Revenue_Growth_Gap_vs_Sector": 0
})


# Save updated dataset

output_path = "/content/Companies_with_Stability_SectorMetrics.xlsx"
df.to_excel(output_path, index=False)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# 1. Load dataset
file_path = "/content/Companies_with_Stability_SectorMetrics.xlsx"
df = pd.read_excel(file_path)

# 2. Define metric groups
growth_metrics = ["Revenue_QoQ_Growth", "Revenue_YoY_Growth",
                  "EPS_QoQ_Growth", "EPS_YoY_Growth"]

profitability_metrics = ["EBITDA_Margin_%", "Net_Profit_Margin_%", "Operating Margin %"]

risk_metrics = ["Interest_Coverage", "PBT_Interest_Ratio", "Debt_Proxy_%"]

stability_metrics = ["Revenue_QoQ_Volatility", "EPS_QoQ_Volatility"]

sector_metrics = ["EBITDA_Margin_Gap_vs_Sector", "Revenue_Growth_Gap_vs_Sector"]

# 3. Data cleaning
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(0, inplace=True)

# Cap extreme outliers for ratios
for col in ["Interest_Coverage", "PBT_Interest_Ratio"]:
    cap = df[col].quantile(0.99)
    df[col] = np.where(df[col] > cap, cap, df[col])

# 4. MinMax Scaling (0-100)
all_metrics = growth_metrics + profitability_metrics + risk_metrics + stability_metrics + sector_metrics
scaler = MinMaxScaler(feature_range=(0, 100))

scaled_values = scaler.fit_transform(df[all_metrics])
scaled_df = pd.DataFrame(scaled_values, columns=[m+"_scaled" for m in all_metrics], index=df.index)

df = pd.concat([df, scaled_df], axis=1)

# 5. Invert "lower=better" metrics
for m in ["Debt_Proxy_%", "Revenue_QoQ_Volatility", "EPS_QoQ_Volatility"]:
    df[m+"_scaled_inv"] = 100 - df[m+"_scaled"]

# Update metric lists with scaled versions
growth_scaled = [m+"_scaled" for m in growth_metrics]
profitability_scaled = [m+"_scaled" for m in profitability_metrics]
risk_scaled = ["Interest_Coverage_scaled", "PBT_Interest_Ratio_scaled", "Debt_Proxy_%_scaled_inv"]
stability_scaled = ["Revenue_QoQ_Volatility_scaled_inv", "EPS_QoQ_Volatility_scaled_inv"]
sector_scaled = [m+"_scaled" for m in sector_metrics]

# 6. Composite scores (0-100 scale) using formulas

# Growth Score (weights: QoQ > YoY)
df["Growth_Score"] = (
    0.3 * df["Revenue_QoQ_Growth_scaled"] +
    0.2 * df["Revenue_YoY_Growth_scaled"] +
    0.3 * df["EPS_QoQ_Growth_scaled"] +
    0.2 * df["EPS_YoY_Growth_scaled"]
)

# Profitability Score (weights: Net Profit > EBITDA > Operating)
df["Profitability_Score"] = (
    0.35 * df["EBITDA_Margin_%_scaled"] +
    0.40 * df["Net_Profit_Margin_%_scaled"] +
    0.25 * df["Operating Margin %_scaled"]
)

# Risk Score (weights: Debt Proxy > Interest Coverage > PBT/Interest)
df["Risk_Score"] = (
    0.5 * df["Debt_Proxy_%_scaled_inv"] +
    0.3 * df["Interest_Coverage_scaled"] +
    0.2 * df["PBT_Interest_Ratio_scaled"]
)

# Stability Score (equal weight)
df["Stability_Score"] = (
    0.5 * df["Revenue_QoQ_Volatility_scaled_inv"] +
    0.5 * df["EPS_QoQ_Volatility_scaled_inv"]
)

# Sector Score (equal weight)
df["Sector_Score"] = (
    0.5 * df["EBITDA_Margin_Gap_vs_Sector_scaled"] +
    0.5 * df["Revenue_Growth_Gap_vs_Sector_scaled"]
)

# Overall Score (equal weight of 5 groups)
df["Overall_Score"] = df[[
    "Growth_Score", "Profitability_Score", "Risk_Score", "Stability_Score", "Sector_Score"
]].mean(axis=1)


# Overall score (equal weight of group scores)
df["Overall_Score"] = df[["Growth_Score", "Profitability_Score",
                          "Risk_Score", "Stability_Score", "Sector_Score"]].mean(axis=1)

# Save output
output_path = "/content/Companies_with_Composite_Scores_0to100.xlsx"
df.to_excel(output_path, index=False)

output_path


'/content/Companies_with_Composite_Scores_0to100.xlsx'

In [ ]:
# Columns to drop
drop_cols = [
    "Revenue_Growth","Expense_Growth","Profit_Growth","Expense_to_Revenue",
    "Revenue_Growth_z","Expense_Growth_z","Profit_Growth_z","Expense_to_Revenue_z",
    "GST_z","Inflation%_z","RepoRate%_z","USDINR_Close_z","Risk",
    "Year","Revenue_QoQ_Growth","Revenue_YoY_Growth","EPS_QoQ_Growth","EPS_YoY_Growth",
    "Frequency","EBITDA_Margin_%","Net_Profit_Margin_%","Interest_Coverage","PBT_Interest_Ratio",
    "Debt_Proxy_%","Revenue_QoQ_Volatility","EPS_QoQ_Volatility","Sector_Median_EBITDA_Margin",
    "EBITDA_Margin_Gap_vs_Sector","Sector_Median_Revenue_QoQ_Growth","Revenue_Growth_Gap_vs_Sector",
    "Revenue_QoQ_Growth_scaled","Revenue_YoY_Growth_scaled","EPS_QoQ_Growth_scaled",
    "EPS_YoY_Growth_scaled","EBITDA_Margin_%_scaled","Net_Profit_Margin_%_scaled",
    "Operating Margin %_scaled","Interest_Coverage_scaled","PBT_Interest_Ratio_scaled",
    "Debt_Proxy_%_scaled","Revenue_QoQ_Volatility_scaled","EPS_QoQ_Volatility_scaled",
    "EBITDA_Margin_Gap_vs_Sector_scaled","Revenue_Growth_Gap_vs_Sector_scaled",
    "Debt_Proxy_%_scaled_inv","Revenue_QoQ_Volatility_scaled_inv","EPS_QoQ_Volatility_scaled_inv"
]

# Drop unwanted columns
df_final = df.drop(columns=drop_cols, errors="ignore")

# Save to new Excel file
final_output_path = "/content/Companies_Final_Composite_Scores.xlsx"
df_final.to_excel(final_output_path, index=False)

final_output_path


'/content/Companies_Final_Composite_Scores.xlsx'